### Magic Formula Implementation 

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
tickers = ["MMM","AXP","AAPL","BA","CAT","CVX","CSCO","KO","DIS","DWDP",
           "XOM","GE","GS","HD","IBM","INTC","JNJ","JPM","MCD","MRK",
           "MSFT","NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT"]

In [18]:
financial_dir = {}

for ticker in tickers:
    try:
        print("Scraping financial data statement data for ", ticker)
        # getting balance sheet data
        url = "https://stockrow.com/api/companies/{}/financials.xlsx?dimension=A&section=Balance%20Sheet&sort=desc".format(ticker)
        df1 = pd.read_excel(url)
        # getting income statement data
        url = "https://stockrow.com/api/companies/{}/financials.xlsx?dimension=A&section=Income%20Statement&sort=desc".format(ticker)
        df2 = pd.read_excel(url)        
        # getting cashflow statement data
        url = "https://stockrow.com/api/companies/{}/financials.xlsx?dimension=A&section=Cash%20Flow&sort=desc".format(ticker)
        df3 = pd.read_excel(url)

        #getting key statistics data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        headers = {'User-Agent':"Chrome/104.0.5112.110"}
        page = requests.get(url, headers=headers)
        page_content = page.content
        soup = BeautifulSoup(page_content, 'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c)"})
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2]) > 0:
                    temp_dir[row.get_text(separator='|').split("|")[0]] = row.get_text(separator='|').split("|")[-1]
        df4 = pd.DataFrame(temp_dir.items(), columns=df3.columns[0:2])
        df4.iloc[:,1] = df4.iloc[:,1].replace({'M': 'E+03', 'B': 'E+06', 'T': 'E+09', '%': 'E-02'}, regex=True)
        df4.iloc[:,1] = pd.to_numeric(df4.iloc[:,1], errors="coerce")
        df4 = df4[df4["Unnamed: 0"].isin(["Market Cap (intraday)", "Forward Annual Dividend Yield"])]

        #combining all extracted information with the corresponding ticker
        df = pd.concat([df1, df2, df3, df4]).iloc[:,[0,1]]
        columns = df.columns.values

        for i in range(len(columns)):
            if columns[i] == "Unnamed: 0":
                columns[i] = "heading"
            else:
                columns[i] = columns[i].strftime("%Y-%m-%d")
        df.columns = columns
        df.set_index("heading", inplace=True)
        financial_dir[ticker] = df
    except Exception as e:
        print(ticker, ":", e)

Scraping financial data statement data for  MMM
Scraping financial data statement data for  AXP
Scraping financial data statement data for  AAPL
Scraping financial data statement data for  BA
Scraping financial data statement data for  CAT
Scraping financial data statement data for  CVX
Scraping financial data statement data for  CSCO
Scraping financial data statement data for  KO
Scraping financial data statement data for  DIS
Scraping financial data statement data for  DWDP
DWDP : HTTP Error 404: Not Found
Scraping financial data statement data for  XOM
Scraping financial data statement data for  GE
Scraping financial data statement data for  GS
Scraping financial data statement data for  HD
Scraping financial data statement data for  IBM
Scraping financial data statement data for  INTC
Scraping financial data statement data for  JNJ
Scraping financial data statement data for  JPM
Scraping financial data statement data for  MCD
Scraping financial data statement data for  MRK
Scraping

In [19]:
# sample
financial_dir['V'].index.values

array(['Cash and Short Term Investments', 'Receivables',
       'Other current assets', 'Total current assets',
       'Property, Plant, Equpment (Net)', 'Long-Term Investments',
       'Goodwill and Intangible Assets (Total)',
       'Long-term assets (Other)', 'Total non-current assets',
       'Total Assets', 'Accounts Payable', 'Dividends Payable',
       'Accrued Expenses', 'Current Part of Debt',
       'Other current liabilities', 'Total current liabilities',
       'Long Term Debt (Total)', 'Long Term Tax Liability (Deferred)',
       'Non-current Liabilities (Other)', 'Total non-current liabilities',
       'Total liabilities', 'Additional Paid In Capital',
       'Preferred Stock (Total)', 'Retained Earnings', 'Treasury Stock',
       'Shareholders Equity (Other)', 'Common Equity (Total)',
       'Shareholders Equity (Total)',
       'Shareholders Equity and Liabilities (Total)', 'Shares (Common)',
       'Shares (Preferred)', 'Shareholders Equity (Tangible)', 'Net Debt',
   

In [20]:
# Creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & Amortization",
         "Net Cash/Marketcap",
         "Net Income Common",
         "Operating Cash Flow",
         "Capital expenditures",
         "Total current assets",
         "Total current liabilities",
         "Property, Plant, Equpment (Net)",
         "Shareholders Equity (Total)",
         "Long Term Debt (Total)",
         "Dividends Paid (Total)"] # change as required

indx = ["EBITDA", "D&A", "MarketCap", "NetIncome", "CashFlowOps", "Capex", "CurrAsset", "CurrLiab", "PPE", "BookValue", "TotDebt", "DivYield"]

def info_filter(df, stats, indx):
    """ function to filter relevant financial information 
    df = dataframe to be filtered
    stats = headings to filter
    indx = rename long headings
    lookback = number of years of data to be retained"""
    for stat in stats:
        if stat not in df.index:
            return
    df_new = df.loc[stats,:]
    df_new.rename(dict(zip(stats, indx)), inplace=True)
    return df_new

In [21]:
# applying filtering to the financials and calculating relevant financial metrics for each stock
transformed_df = {}
for ticker in financial_dir:
    transformed_df[ticker] = info_filter(financial_dir[ticker], stats, indx)
    if transformed_df[ticker] is None:
        del transformed_df[ticker]
        continue
    transformed_df[ticker].loc["EBIT",:] = transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:]
    transformed_df[ticker].loc["TEV",:] = transformed_df[ticker].loc["MarketCap",:] + transformed_df[ticker].loc["TotDebt",:] - (transformed_df[ticker].loc["CurrAsset",:] - transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["EarningYield",:] = transformed_df[ticker].loc["EBIT",:]/transformed_df[ticker].loc["TEV",:]
    transformed_df[ticker].loc["FCFYield",:] = (transformed_df[ticker].loc["CashFlowOps",:] - transformed_df[ticker].loc["Capex",:])/transformed_df[ticker].loc["MarketCap",:]
    transformed_df[ticker].loc["ROC",:] = (transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:]) / (transformed_df[ticker].loc["PPE",:] + transformed_df[ticker].loc["CurrAsset",:] - transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["BookToMkt",:] = transformed_df[ticker].loc["BookValue",:] / transformed_df[ticker].loc["MarketCap",:]

In [22]:
## Output DataFrame
final_stats_val_df = pd.DataFrame(columns=transformed_df.keys())
for key in transformed_df:
    final_stats_val_df[key] = transformed_df[key].values.flatten()
    final_stats_val_df.set_index(transformed_df[key].index, inplace=True)

In [23]:
# Finding value stocks based on Greenblatt Magic Formula
final_stats_val_df.loc["CombRank", :] = final_stats_val_df.loc["EarningYield", :].rank(ascending=False, na_option='bottom') + final_stats_val_df.loc["ROC", :].rank(ascending=False, na_option='bottom')
final_stats_val_df.loc["MagicFormulaRank", :] = final_stats_val_df.loc["CombRank", :].rank(method='first')
value_stocks = final_stats_val_df.loc["MagicFormulaRank", :].sort_values()

In [24]:
print("Value Stocks based on Greenblatt's Magic Formula")
print(value_stocks)

Value Stocks based on Greenblatt's Magic Formula
AAPL     1.0
V        2.0
JNJ      3.0
PFE      4.0
PG       5.0
HD       6.0
MMM      7.0
INTC     8.0
IBM      9.0
KO      10.0
MRK     11.0
CAT     12.0
CVX     13.0
CSCO    14.0
XOM     15.0
WMT     16.0
VZ      17.0
MCD     18.0
GE      19.0
MSFT    20.0
NKE     21.0
UNH     22.0
DIS     23.0
GS      24.0
BA      25.0
Name: MagicFormulaRank, dtype: float64


In [25]:
# Finding highest dividend yield stocks
high_dividend_stocks = final_stats_val_df.loc["DivYield",:].sort_values(ascending=False)
print("Highest Dividend Paying Stocks")
print(high_dividend_stocks)

Highest Dividend Paying Stocks
GE     -5.750000e+08
NKE    -1.837000e+09
CAT    -2.332000e+09
GS     -2.725000e+09
V      -2.798000e+09
MMM    -3.420000e+09
MCD    -3.919000e+09
UNH    -5.280000e+09
INTC   -5.644000e+09
IBM    -5.869000e+09
WMT    -6.152000e+09
CSCO   -6.224000e+09
MRK    -6.610000e+09
HD     -6.985000e+09
KO     -7.252000e+09
PFE    -8.729000e+09
PG     -8.770000e+09
CVX    -1.021500e+10
VZ     -1.044500e+10
JNJ    -1.103200e+10
AAPL   -1.446700e+10
XOM    -1.514800e+10
MSFT   -1.813500e+10
BA               NaN
DIS              NaN
Name: DivYield, dtype: float64


In [29]:
final_stats_val_df.T

heading,EBITDA,D&A,MarketCap,NetIncome,CashFlowOps,Capex,CurrAsset,CurrLiab,PPE,BookValue,...,DivYield,EBIT,TEV,EarningYield,FCFYield,ROC,BookToMkt,CombRank,MagicFormulaRank,CombinedRank
MMM,9.441000e+09,1.915000e+09,0.1327,5.921000e+09,7.454000e+09,-1.552000e+09,1.540300e+10,9.035000e+09,9.429000e+09,1.511700e+10,...,-3.420000e+09,7.526000e+09,9.688000e+09,0.776837,6.786737e+10,0.476420,1.139186e+11,19.0,7.0,25.0
AAPL,1.202330e+11,1.128400e+10,0.0317,9.468000e+10,1.040380e+11,-1.108500e+10,1.348360e+11,1.254810e+11,4.952700e+10,6.309000e+10,...,-1.446700e+10,1.089490e+11,1.100260e+11,0.990211,3.631640e+12,1.850294,1.990221e+12,6.0,1.0,27.0
BA,-7.260000e+08,2.144000e+09,0.3654,-4.202000e+09,-3.416000e+09,-4.510000e+08,1.086660e+11,8.199200e+10,1.235500e+10,-1.499900e+10,...,NaN,-2.870000e+09,3.140300e+10,-0.091393,-8.114395e+09,-0.073535,-4.104817e+10,45.0,25.0,NaN
CAT,9.320000e+09,2.352000e+09,0.2618,6.489000e+09,7.198000e+09,-1.207000e+09,4.345500e+10,2.984700e+10,1.209000e+10,1.651600e+10,...,-2.332000e+09,6.968000e+09,1.242500e+10,0.560805,3.210466e+10,0.271150,6.308633e+10,23.0,12.0,26.0
CVX,4.027600e+10,1.792500e+10,0.1143,1.562500e+10,2.918700e+10,-8.056000e+09,3.373800e+10,2.679100e+10,1.469610e+11,1.399400e+11,...,-1.021500e+10,2.235100e+10,2.416600e+10,0.924894,3.258355e+11,0.145223,1.224322e+12,26.0,13.0,44.0
CSCO,1.592600e+10,1.957000e+09,-0.0522,1.181200e+10,1.322600e+10,-3.860000e+08,3.671700e+10,2.564000e+10,1.997000e+09,3.977300e+10,...,-6.224000e+09,1.396900e+10,-2.661000e+09,-5.249530,-2.607663e+11,1.068456,-7.619349e+11,29.0,14.0,41.0
KO,1.176000e+10,1.452000e+09,0.1223,9.771000e+09,1.262500e+10,-1.259000e+09,2.254500e+10,1.995000e+10,9.920000e+09,2.486000e+10,...,-7.252000e+09,1.030800e+10,3.552100e+10,0.290195,1.135241e+11,0.823652,2.032706e+11,21.0,10.0,36.0
DIS,8.852000e+09,5.111000e+09,0.1377,1.995000e+09,5.566000e+09,-3.578000e+09,3.147400e+10,3.107700e+10,3.685500e+10,8.855300e+10,...,NaN,3.741000e+09,5.137200e+10,0.072822,6.640523e+10,0.100424,6.430864e+11,43.0,23.0,NaN
XOM,4.839600e+10,2.060700e+10,0.1767,2.304000e+10,4.812900e+10,-1.207600e+10,5.915400e+10,5.664300e+10,2.165520e+11,1.685770e+11,...,-1.514800e+10,2.778900e+10,4.487100e+10,0.619309,3.407187e+11,0.126854,9.540294e+11,29.0,15.0,51.0
GE,7.465000e+09,3.009000e+09,0.0960,-2.911000e+09,3.332000e+09,-1.083000e+09,6.634800e+10,5.195300e+10,1.560900e+10,4.031000e+10,...,-5.750000e+08,4.456000e+09,1.927700e+10,0.231156,4.598958e+10,0.148514,4.198958e+11,35.0,19.0,36.0


In [31]:
# Magic formula and Dividend Yield Combined
final_stats_val_df.loc["CombRank",:] = final_stats_val_df.loc["EarningYield",:].rank(ascending=False, method='first') + final_stats_val_df.loc["ROC",:].rank(ascending=False, method='first') + final_stats_val_df.loc["DivYield",:].rank(ascending=False, method='first')
value_high_div_stocks = final_stats_val_df.T.loc[:, ["EarningYield", "ROC", "DivYield", "CombinedRank"]].sort_values("CombinedRank")
print("Magic Formula and Dividend Yield Combined")
print(value_high_div_stocks.rank(method='first'))

Magic Formula and Dividend Yield Combined
heading  EarningYield   ROC  DivYield  CombinedRank
V                24.0  22.0      19.0           1.0
MMM              19.0  14.0      18.0           2.0
CAT              17.0  12.0      21.0           3.0
AAPL             22.0  24.0       3.0           4.0
INTC             25.0   8.0      15.0           5.0
PFE              21.0  17.0       8.0           6.0
IBM               7.0  25.0      14.0           7.0
HD               16.0  19.0      10.0           8.0
JNJ              23.0  18.0       4.0           9.0
PG               14.0  23.0       7.0          10.0
MRK              15.0  15.0      11.0          11.0
KO               11.0  20.0       9.0          12.0
GE               10.0   7.0      23.0          13.0
NKE               3.0  11.0      22.0          14.0
WMT              13.0  10.0      13.0          15.0
CSCO              2.0  21.0      12.0          16.0
MCD               9.0   9.0      17.0          17.0
CVX              20.0 